In [1]:
import pandas as pd
import numpy as np

from os import listdir
from os.path import isfile, join

from rapidfuzz import process

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder

In [2]:
datasets = {}

path = "data/"
for f in listdir(path):
    if isfile(join(path, f)):
        datasets[f.replace(".csv", "")] = pd.read_csv(join(path, f))

print(datasets.keys())

dict_keys(['game_lineups', 'stats', 'Soccer_Football Clubs Ranking', 'players', 'club_games', 'clubs'])


player_valuations: Les joueurs ont leur prix évalués chaque année, 2 fois, dans le dataset.
Hors, la dernière année comporte qu'une valeur. Cette valeur correspond a celle du dataset players.csv
On peut donc se passer de player_valuations.

In [3]:
# Clear columns
clear_columns = {
    "players": {
        "first_name",
        "last_name",
        "player_code",
        "city_of_birth",
        "country_of_citizenship",
        "image_url",
        "url"
    },

    "stats": {
        "Rk", "Born", 
    }
}

# Clear too old data, we keep 2023 ones.
# The issue is that, we have valuations for each year. Something we don't want.

# Some names are in double in stats.csv, keep the newer


Players et stats sont 2 datasets fusionnés
Ce qu'on peut extraire des autres csv est le ratio de victoires ( club_games ), combien de fois le joueur a été capitaine, si il était remplaçant ou titulaire ( game_lineups ),  

In [4]:
lups = datasets["game_lineups"]
all_players = lups['player_id'].unique()

# capitaines
team_captain_games = lups[lups['team_captain'] == 1]
captain_counts = team_captain_games['player_id'].value_counts()
captain_counts = captain_counts.reindex(all_players, fill_value=0)

# starting
starting_lineups = lups[lups['type'] == "starting_lineup"]
starting_counts = starting_lineups['player_id'].value_counts()
starting_counts = starting_counts.reindex(all_players, fill_value=0)

# substitutes
substitutes = lups[lups['type'] == "substitutes"]
substitutes_counts = substitutes['player_id'].value_counts()
substitutes_counts = substitutes_counts.reindex(all_players, fill_value=0)

# Rename
substitutes_counts.columns = ['player_id', 'substitutes_number']
starting_counts.columns = ['player_id', 'starting_lineup_number']
captain_counts.columns = ['player_id', 'captain_number']

starting_counts = starting_counts.reset_index()
substitutes_counts = substitutes_counts.reset_index()
captain_counts = captain_counts.reset_index()

# Merge the clean_dataset with counts DataFrames
_datasetCleaning = pd.merge(pd.merge(datasets["stats"], datasets["players"], on="name"), substitutes_counts, on="player_id", how="left")
_datasetCleaning = pd.merge(_datasetCleaning, starting_counts, on="player_id", how="left")
_datasetCleaning = pd.merge(_datasetCleaning, captain_counts, on="player_id", how="left")

In [5]:
print(_datasetCleaning.shape)

(1933, 149)


1.? - 1-Hot encoding/String transform

In [6]:
_datasetCleaning["foot"] = [int(x == "right") for x in _datasetCleaning["foot"]]
_datasetCleaning = pd.concat([_datasetCleaning, pd.get_dummies(_datasetCleaning.position)], axis=1)
_datasetCleaning = pd.concat([_datasetCleaning, pd.get_dummies(_datasetCleaning.sub_position)], axis=1)
_datasetCleaning = pd.concat([_datasetCleaning, pd.get_dummies(_datasetCleaning.Comp)], axis=1)

In [7]:
_datasetCleaning.head()

,Rk,name,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Left-Back,Right Midfield,Right Winger,Right-Back,Second Striker,Bundesliga,La Liga,Ligue 1,Premier League,Serie A
0,1,Brenden Aaronson,USA,MFFW,Leeds United,Premier League,22,2000,20,19,...,False,False,False,False,False,False,False,False,True,False
1,2,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,35,1987,22,22,...,False,False,False,False,False,False,False,True,False,False
2,3,Himad Abdelli,FRA,MFFW,Angers,Ligue 1,23,1999,14,8,...,False,False,False,False,False,False,False,True,False,False
3,4,Salis Abdul Samed,GHA,MF,Lens,Ligue 1,22,2000,20,20,...,False,False,False,False,False,False,False,True,False,False
4,5,Laurent Abergel,FRA,MF,Lorient,Ligue 1,30,1993,15,15,...,False,False,False,False,False,False,False,True,False,False


### Converting squad column into club ranking

Since the squad column is a categorical value with many possible outcomes, performing one-hot encoding on it would produce a big amount of columns (one for each team).

We know there should be a correlation between the club a player belongs to and a player's market value. Since good teams usually have good players, so more valuable ones. Therefore, we would like to keep some information about the team. 

In order to solve this problem, we use another dataset in which we have several football teams and their latest rankings, and we try to replace the club names with their rankings.

The first problem we encounter is that the clubs in our original dataset might be refered to with slightly different names than in the new dataset we're trying to use, since we got it from a different source, for example, 'Machester United' could be referred to as 'Man United'.

To resolve this, we use fuzzy matching to find, for each player’s club, the closest matching club name in the rankings dataset. Once the matches are established, we replace the club names in the original dataset with their corresponding rankings. This method works because the club names in both datasets are very similar and differ only by minor variations, such as spelling or abbreviations.

First, we notice that all the players we have in the original dataset play in the top five leagues, therefore we restrict the rankings dataset to teams from five countries (England, Spain, Germany, Italy and France). This is especially useful because some teams have the same name. For example, 'Real Sociedad' is both present in Spain and Uruguay. But we make sure that there are no teams with the same name from the rankings dataset in these five countries. This way we make sure the matching is done correctly.

In [8]:
# Players outside of the top 5 leagues in our dataset

s = 0
leagues = _datasetCleaning.iloc[:, -5:]
indexes = []
for i, row in leagues.iterrows():
    add = False
    for j in range(5):
        if row[j] == True:
            add = True
    if not add:
        s += 1
        indexes.append(i)
s

/var/folders/9k/jzxy5b0j6zjcx9kq2ns60_f80000gn/T/ipykernel_87059/1870972108.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if row[j] == True:


0

We just checked that there are no players out of the top five leagues, so we can proceed with restricting the rankings dataset on the five countries mentioned previously. 

In [9]:
rankings_df = pd.read_csv(path + 'Soccer_Football Clubs Ranking.csv', sep=';')

# We only keep teams in the big 5 leagues
rankings_df = rankings_df[rankings_df['country'].isin(['Germany', 'Spain', 'France', 'Italy', 'England'])]
rankings_df.head()

,ranking,club name,country,point score,1 year change,previous point scored,symbol change
0,88,1. FC Köln,Germany,1633,105,1545,+
1,74,1. FC Union Berlin,Germany,1655,5,1654,+
20,11,AC Milan,Italy,1850,21,1741,+
54,270,AFC Bournemouth,England,1512,7,1512,-
70,823,Ajaccio,France,1369,13,1369,-


We check for duplicates

In [10]:
# Check for duplicate teams
duplicates = rankings_df['club name '][rankings_df['club name '].duplicated()]

if not duplicates.empty:
    print(f"There are {duplicates.nunique()} unique duplicate values and {duplicates.size} total duplicates in the column 'club name '.")
    print("Duplicate values and their counts:")
    print(duplicates.value_counts())
else:
    print(f"There are no duplicates in the column 'club name '.")

There are 6 unique duplicate values and 6 total duplicates in the column 'club name '.
Duplicate values and their counts:
club name 
Angers           1
Evian TG         1
GFC Ajaccio      1
Málaga           1
Middlesbrough    1
Nancy            1
Name: count, dtype: int64


In [11]:
duplicate_rows = rankings_df[rankings_df['club name '].duplicated(keep=False)]
duplicate_rows

,ranking,club name,country,point score,1 year change,previous point scored,symbol change
223,475,Angers,France,1440,121,1474,-
224,475,Angers,France,1440,121,1474,-
913,484,Evian TG,France,1439,2,1439,-
914,484,Evian TG,France,1439,2,1439,-
1165,485,GFC Ajaccio,France,1438,1,1438,-
1166,485,GFC Ajaccio,France,1438,1,1438,-
1628,451,Málaga,Spain,1446,1,1446,-
1629,451,Málaga,Spain,1446,1,1446,-
1691,465,Middlesbrough,England,1443,4,1443,-
1692,465,Middlesbrough,England,1443,4,1443,-


We notice that the rows don't refer to different teams, so we can delete them.

In [12]:
# Remove duplicate rows based on the column 'club name '
rankings_df = rankings_df.drop_duplicates(subset='club name ', keep='first')
duplicates = rankings_df['club name '][rankings_df['club name '].duplicated()]

if not duplicates.empty:
    print(f"There are {duplicates.nunique()} unique duplicate values and {duplicates.size} total duplicates in the column 'club name '.")
    print("Duplicate values and their counts:")
    print(duplicates.value_counts())
else:
    print(f"There are no duplicates in the column 'club name '.")

There are no duplicates in the column 'club name '.


Now, we can proceed with our matching:

In [13]:
# Create a dictionary of matches
club_list = rankings_df['club name '].tolist()
club_rankings = rankings_df.set_index('club name ')['ranking'].to_dict()

def get_best_match(club_name):
    match, score, _ = process.extractOne(club_name, club_list)
    if score >= 50:  # Adjust threshold as needed
        return club_rankings[match]
    return None

# Apply the function to all rows using pandas' `apply`
_datasetCleaning['Squad'] = _datasetCleaning['Squad'].apply(get_best_match)
_datasetCleaning.rename(columns={'Squad': 'Club Ranking'}, inplace=True)


In [14]:
_datasetCleaning.head()

,Rk,name,Nation,Pos,Club Ranking,Comp,Age,Born,MP,Starts,...,Left-Back,Right Midfield,Right Winger,Right-Back,Second Striker,Bundesliga,La Liga,Ligue 1,Premier League,Serie A
0,1,Brenden Aaronson,USA,MFFW,162,Premier League,22,2000,20,19,...,False,False,False,False,False,False,False,False,True,False
1,2,Yunis Abdelhamid,MAR,DF,203,Ligue 1,35,1987,22,22,...,False,False,False,False,False,False,False,True,False,False
2,3,Himad Abdelli,FRA,MFFW,475,Ligue 1,23,1999,14,8,...,False,False,False,False,False,False,False,True,False,False
3,4,Salis Abdul Samed,GHA,MF,199,Ligue 1,22,2000,20,20,...,False,False,False,False,False,False,False,True,False,False
4,5,Laurent Abergel,FRA,MF,382,Ligue 1,30,1993,15,15,...,False,False,False,False,False,False,False,True,False,False


Now, the clubs are replaced with their corresponding rankings. We can proceed with the data cleaning.

1.? - Nettoyage

We convert boolean columns to 0s and 1s.

In [15]:
bool_columns = _datasetCleaning.select_dtypes(include=['bool']).columns
for col in bool_columns:
    _datasetCleaning[col] = _datasetCleaning[col].astype(int)

print("Boolean columns converted to 0s and 1s.")

Boolean columns converted to 0s and 1s.


We drop non-numeric column we don't need from our dataset.

In [16]:
# Identify non-numeric columns in the dataset
post_process_drops = [
    "name", "Nation", "first_name", "last_name", "player_code", "city_of_birth", "date_of_birth", "contract_expiration_date", "agent_name", "image_url", 
    'current_club_domestic_competition_id', "url",
    "country_of_birth", "country_of_citizenship", # 102 valeurs uniques
    "sub_position", "position", "Comp", # 1-Hot encoding
    "current_club_name", # 198 valeurs uniques
    "Pos", # Doublon
    "Born" # The year in which a player is equivalent the age so we remove it
]

_datasetCleaning = _datasetCleaning.drop(post_process_drops, axis=1, errors='ignore')

In [17]:
# Check for non-numeric columns
non_numeric_columns = _datasetCleaning.select_dtypes(exclude=['number'])
if not non_numeric_columns.empty:
    print("Non-numeric columns and their types:")
    for col in non_numeric_columns.columns:
        print(f"{col}: {non_numeric_columns[col].dtype}")
else:
    print("No non-numeric columns found.")

No non-numeric columns found.


Cleaning null values:

In [18]:
# Cleaning null values
columnsSizePreNA = _datasetCleaning.shape[1]
_datasetCleaning.dropna(how='all', axis=1, inplace=True) # On retire les colonnes inutiles
print("On a retiré " + str(columnsSizePreNA - _datasetCleaning.shape[1]) + " colonnes.")

columnsSizePreNA = _datasetCleaning.shape[0]
_datasetCleaning.dropna(how='any', axis=0, inplace=True) # Et on retire les valeurs nulles
print("On a retiré " + str(columnsSizePreNA - _datasetCleaning.shape[0]) + " entrées.")

On a retiré 0 colonnes.
On a retiré 372 entrées.


In [19]:
_datasetCleaning.head()

,Rk,Club Ranking,Age,MP,Starts,Min,90s,Goals,Shots,SoT,...,Left-Back,Right Midfield,Right Winger,Right-Back,Second Striker,Bundesliga,La Liga,Ligue 1,Premier League,Serie A
0,1,162,22,20,19,1596,17.7,1,1.53,0.28,...,0,0,0,0,0,0,0,0,1,0
1,2,203,35,22,22,1980,22.0,0,0.86,0.05,...,0,0,0,0,0,0,0,1,0,0
3,4,199,22,20,20,1799,20.0,1,0.60,0.15,...,0,0,0,0,0,0,0,1,0,0
4,5,382,30,15,15,1165,12.9,0,0.31,0.00,...,0,0,0,0,0,0,0,1,0,0
5,6,79,26,1,0,11,0.1,0,0.00,0.00,...,0,0,0,0,0,0,0,0,0,1


2 - Préparation du modèle

In [20]:
X = _datasetCleaning.drop('market_value_in_eur', axis=1)
Y = _datasetCleaning["market_value_in_eur"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

3 - Modèle(s)

In [21]:
models = {
    "Linear Regression": (LinearRegression(), {"model__fit_intercept": [True, False]}),
    "Ridge Regression": (Ridge(), {"model__alpha": [0.1, 1.0, 10.0]}),
    "Lasso Regression": (Lasso(), {"model__alpha": [0.01, 0.1, 1.0]}),
    "ElasticNet": (ElasticNet(), {"model__alpha": [0.01, 0.1, 1.0], "model__l1_ratio": [0.3, 0.5, 0.7]}),
    "Decision Tree": (DecisionTreeRegressor(), {"model__max_depth": [3, 5, 10], "model__min_samples_split": [2, 5, 10]}),
    "Random Forest": (RandomForestRegressor(random_state=42), {"model__n_estimators": [50, 100, 200], "model__max_depth": [3, 5, 10]}),
    "Gradient Boosting": (GradientBoostingRegressor(random_state=42), {"model__n_estimators": [50, 100, 200], "model__learning_rate": [0.01, 0.1, 0.2], "model__max_depth": [3, 5]})
}

# Dictionary to store results
results = {}

# Train and evaluate each model
for name, (model, params) in models.items():
    print(f"Training {name}...")
    
    # Create the pipeline with standard scaler and the model
    pipeline = Pipeline([
        ("scaler", StandardScaler()),
        ("model", model)
    ])
    
    # Perform GridSearchCV
    grid = GridSearchCV(
        pipeline, 
        param_grid=params, 
        cv=5, 
        scoring="neg_mean_squared_error", 
        n_jobs=-1
    )
    grid.fit(X_train, y_train)
    
    # Evaluate on test set
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Store results
    results[name] = {
        "Best Params": grid.best_params_,
        "Best CV Score (MSE)": -grid.best_score_,
        "Test MSE": mse,
        "Test R^2": r2,
        "Best Model": best_model
    }

# Print summary of results
for name, res in results.items():
    print(f"\n{name}")
    print(f"Best Params: {res['Best Params']}")
    print(f"Best CV Score (MSE): {res['Best CV Score (MSE)']:.2f}")
    print(f"Test MSE: {res['Test MSE']:.2f}")
    print(f"Test R^2: {res['Test R^2']:.2f}")

Training Linear Regression...
Training Ridge Regression...
Training Lasso Regression...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.484e+16, tolerance: 2.294e+13
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.376e+16, tolerance: 2.277e+13
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

Training ElasticNet...


/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.562e+16, tolerance: 2.294e+13
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.454e+16, tolerance: 2.277e+13
  model = cd_fast.enet_coordinate_descent(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/linear_model/_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iter

Training Decision Tree...
Training Random Forest...
Training Gradient Boosting...

Linear Regression
Best Params: {'model__fit_intercept': True}
Best CV Score (MSE): 89967632933557.47
Test MSE: 105166686870393.25
Test R^2: 0.71

Ridge Regression
Best Params: {'model__alpha': 10.0}
Best CV Score (MSE): 83333152786117.55
Test MSE: 96674426424924.52
Test R^2: 0.74

Lasso Regression
Best Params: {'model__alpha': 1.0}
Best CV Score (MSE): 87835033803756.14
Test MSE: 100174936641220.27
Test R^2: 0.73

ElasticNet
Best Params: {'model__alpha': 0.1, 'model__l1_ratio': 0.3}
Best CV Score (MSE): 78547649379153.36
Test MSE: 98589951175844.84
Test R^2: 0.73

Decision Tree
Best Params: {'model__max_depth': 5, 'model__min_samples_split': 10}
Best CV Score (MSE): 85286395474424.09
Test MSE: 109089571126794.62
Test R^2: 0.70

Random Forest
Best Params: {'model__max_depth': 10, 'model__n_estimators': 200}
Best CV Score (MSE): 46237431963823.25
Test MSE: 76423318874420.27
Test R^2: 0.79

Gradient Boostin